# CallAPI

In [1]:
import time
from googleapiclient.discovery import build
import isodate
import json
import os

# 환경 변수에서 API 키 가져오기
API_KEY = os.getenv("YOUTUBE_API_KEY")
if not API_KEY:
    raise ValueError("API 키가 설정되지 않았습니다. 환경 변수 'YOUTUBE_API_KEY'를 설정하세요.")

# YouTube API 설정
youtube = build("youtube", "v3", developerKey=API_KEY)

# 카테고리 ID 설정
CATEGORIES = {
    "News & Politics": "25"
}

# 동영상 데이터 가져오기
def fetch_trending_videos(category_id, region_code="KR", max_results=200):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                chart="mostPopular",
                regionCode=region_code,
                videoCategoryId=category_id,
                maxResults=min(50, max_results - len(videos)),
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                duration = isodate.parse_duration(item["contentDetails"]["duration"])
                duration_in_seconds = duration.total_seconds()  #초로 바꾸기기

                if duration_in_seconds > 80:  # 80초 이상의 동영상만 가져오기
                    videos.append({
                        "video_id": item["id"],
                        "title": item["snippet"]["title"],
                        "description": item["snippet"]["description"],
                        "tags": item["snippet"].get("tags", []),
                        "duration": str(duration),
                        "view_count": int(item["statistics"].get("viewCount", 0)),
                        "like_count": int(item["statistics"].get("likeCount", 0)),
                        "comment_count": int(item["statistics"].get("commentCount", 0)),
                        "category_id": category_id,
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

        except Exception as e:
            print(f"Error fetching videos: {e}")
            time.sleep(5)  # 잠시 대기 후 다시 시도

    return videos

# JSON 파일로 저장하는 함수
def save_to_json(data, filename):
    # data 폴더가 존재하지 않으면 생성
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # 파일 경로를 data 폴더 아래로 설정
    filepath = os.path.join('data', filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 실행
all_videos = {}

for category_name, category_id in CATEGORIES.items():
    print(f"Fetching trending videos for category: {category_name}")
    videos = fetch_trending_videos(category_id, region_code="KR", max_results=200)
    all_videos[category_name] = videos
    print(f"비디오 {len(videos)} 개 카테고리: {category_name} fetch 완료.")

# 결과를 하나의 JSON 파일로 저장
output_file = "raw_video_data.json"
save_to_json(all_videos, output_file)
print(f"데이터 저장 : data/{output_file}'")

# 결과 출력 예시
for category, videos in all_videos.items():
    print(f"\nCategory: {category}")
    for video in videos[:5]:
        print(f" - {video['title']} ({video['video_id']}), 조회수: {video['view_count']} 회, 좋아요: {video['like_count']} 개")

Fetching trending videos for category: News & Politics
비디오 60 개 카테고리: News & Politics fetch 완료.
데이터 저장 : data/raw_video_data.json'

Category: News & Politics
 - 김건희와 검찰을 끝까지 의심해야 하는 이유? (곽종근의 분노)｜풀버전 (l_MGS00btCo), 조회수: 928739 회, 좋아요: 55188 개
 - 미친 듯 날뛰던 이재명. 전한길 등장에 꼭꼭 숨었다 [주말 몰아보기] (xJbj4MRPXOs), 조회수: 1032711 회, 좋아요: 89980 개
 - 이상민 입 열자 尹 '휘청' "10시 KBS" 발언의 비밀 [뉴스.zip/MBC뉴스] (EMKYRHDCDC8), 조회수: 1184825 회, 좋아요: 22662 개
 - 尹 사건 맡은 중앙지법! 대형사건 터졌다![배승희 뉴스배송] (xYld5mZwuQU), 조회수: 846646 회, 좋아요: 141685 개
 - “극우가 점입가경”…민주당, 전한길 공개 비판 / 채널A / 김진의 돌직구쇼 (6sk_f6hx0OY), 조회수: 411831 회, 좋아요: 20045 개


In [2]:
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# JSON 파일 경로
input_file_path = "data/raw_video_data.json"

# JSON 데이터 로드
data = load_json(input_file_path)

def merge_text_fields(video):
    title = video.get("title", "")
    description = video.get("description", "")
    tags = " ".join(video.get("tags", []))  # 태그 리스트를 공백으로 연결

    # 병합
    merged_text = " ".join([title, description, tags])
    return merged_text.strip()  # 공백 제거

def extract_texts(data):
    texts = []
    for category, videos in data.items():
        for video in videos:
            merged_text = merge_text_fields(video)
            texts.append(merged_text)
    return texts

# 텍스트 데이터 추출
texts = extract_texts(data)